In [1]:
import glob
import os
import pandas as pd
import jieba

fs = glob.glob("chinese_news_test/*/*[tT][xX][tT]")
cats = []
contents = []
for fname in fs:
    fn = os.path.split(fname)[-1]
    with open(fname, "r", encoding = "utf8") as f:
        content = f.read()
        contents.append(content)
    dn = os.path.split(fname)[0]
    dn = os.path.split(dn)[-1]
    cats.append(dn)
    
test_df = pd.DataFrame({
    "content": contents,
    "ans":cats
}
)

test_df

,content,ans
0,日月光華 -- Traffic_Info精華區文章閱讀\n-----------------...,交通
1,日月光華 -- Traffic_Info精華區文章閱讀\n-----------------...,交通
2,日月光華 -- Traffic_Info精華區文章閱讀\n-----------------...,交通
3,三趟火車停開 乘客可全額退票 \n瀏覽次數：1180 \n 昨日，來自鐵路部門的消息說...,交通
4,日月光華 -- Traffic_Info精華區文章閱讀\n-----------------...,交通
...,...,...
96,最優秀選手無緣亞運會健美賽\n \n \n 健美在亞洲運動會上是“新生兒”——韓國釜...,體育
97,\n各國記者眼中的羽毛球世錦賽\n \n\n-----------------------...,體育
98,\n友好運動會第五天 東道主選手大顯神威\n2001年09月03日02:51:40 新華社 ...,體育
99,不靠技術比運氣\n\n 第二屆奧運會在巴黎舉行，同時這裡也正在舉行國際博覽會，東道主把一些...,體育


In [2]:
fs = glob.glob("chinese_news_trans/*/*[tT][xX][tT]")
cats = []
contents = []
for fname in fs:
    fn = os.path.split(fname)[-1]
    with open(fname, "r", encoding = "utf8") as f:
        content = f.read()
        contents.append(content)
    dn = os.path.split(fname)[0]
    dn = os.path.split(dn)[-1]
    cats.append(dn)
    
train_df = pd.DataFrame({
    "content": contents,
    "ans":cats
}
)

train_df

,content,ans
0,【 日 期 】19960104\n【 版 號 】1\n【 標 題 】合巢蕪高速公路巢蕪...,交通
1,【 日 期 】19960226\n【 版 號 】5\n【 標 題 】我首臺載人磁懸浮列...,交通
2,大秦鐵路萬噸列車試運成功\n新華社北京５月２４日電（通訊員李忠民）大秦...,交通
3,遼寧省檯安縣村村都通柏油路鄉村公路建設全國數第\n一\n (4)...,交通
4,北京—烏蘭巴托—莫斯料３／４次國際旅客列車通車\n３０週年\n ...,交通
...,...,...
2632,馬玉芹破女子４００米跑全國青年紀錄\n新華社鞍山５月１０日電（通訊員王...,體育
2633,國際奧委會中國臺北委員吳經國訪問北京\n新華社北京５月１０日電記者《中...,體育
2634,亞奧理事會３９個成員組織全部以書面形式確認參加\n第十一屆亞運會\n ...,體育
2635,世界盃乒乓球團體賽男子團體採用新賽制\n新華社北京５月１０日電（記者樑...,體育


In [3]:
jieba.set_dictionary('dict.txt.big')
def cut(s):
    s = s.replace("\r", "").replace("\r", "")
    return " ".join(jieba.cut(s))

# apply: pandas operation to the all row
x_train = train_df["content"].apply(cut)
x_test = test_df["content"].apply(cut)

Building prefix dict from D:\Tibame\周凡剛\dict.txt.big ...
Dumping model to file cache C:\Users\CHIENH~1\AppData\Local\Temp\jieba.ub5c3068c02813658e137069135d788c6.cache
Loading model cost 3.601 seconds.
Prefix dict has been built succesfully.


In [4]:
u = train_df["ans"].unique()
trans = {label:i for i, label in enumerate(u)}

# trans = {
#     "交通": 0,
#     "政治": 1,
#     "計算機": 2,
#     "軍事": 3,
#     "教育": 4,
#     "經濟": 5,
#     "環境": 6,
#     "醫藥": 7,
#     "藝術": 8,
#     "體育": 9
# }
reverse_trans = {v:k for k,v in trans.items()}
y_train = train_df["ans"].replace(trans)
y_test = test_df["ans"].replace(trans)

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
# Naive bayes can filter the important variables. Therefore, we don't use TfidfVectorizer
vec = CountVectorizer()
x_train_vec = vec.fit_transform(x_train)
x_test_vec = vec.transform(x_test)
# vec.vocabulary_

In [6]:
from sklearn.naive_bayes import MultinomialNB
# The feature value is discrete.
# GaussianNB: the feature value is continuous.
# BernouliNB: the feature value is binary.
clf = MultinomialNB(alpha = 1)
clf.fit(x_train_vec, y_train)

MultinomialNB(alpha=1, class_prior=None, fit_prior=True)

In [7]:
from sklearn.metrics import accuracy_score
pre = clf.predict(x_test_vec)
accuracy_score(y_test, pre)

1.0

In [8]:
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(y_test, pre)
ls = ["交通", "政治", "計算機", "軍事", "教育", "經濟", "環境", "醫藥", "藝術", "體育"]
i = [n + "(真實)" for n in ls]
c = [n + "(預測)" for n in ls]
pd.DataFrame(mat,
            columns = c,
            index = i)

,交通(預測),政治(預測),計算機(預測),軍事(預測),教育(預測),經濟(預測),環境(預測),醫藥(預測),藝術(預測),體育(預測)
交通(真實),10,0,0,0,0,0,0,0,0,0
政治(真實),0,11,0,0,0,0,0,0,0,0
計算機(真實),0,0,10,0,0,0,0,0,0,0
軍事(真實),0,0,0,10,0,0,0,0,0,0
教育(真實),0,0,0,0,10,0,0,0,0,0
經濟(真實),0,0,0,0,0,10,0,0,0,0
環境(真實),0,0,0,0,0,0,10,0,0,0
醫藥(真實),0,0,0,0,0,0,0,10,0,0
藝術(真實),0,0,0,0,0,0,0,0,10,0
體育(真實),0,0,0,0,0,0,0,0,0,10


In [9]:
p_list = []
p = []
while True:
    a = input("請輸入新聞：")
    if a == "0":
        break
    else:
        p_list.append(a)

for x in range(0, len(p_list)):
    review = cut(p_list[x])
    p.append(review)
    
# p = vec.transform(p).toarray()
p = vec.transform(p)

請輸入新聞：防堵我國陷入社區傳播，中央流行疫情指揮中心發言人莊人祥今天表示，稍早決定，最快從今起，正在執行居家檢疫或是居家隔離者，將推出「加強版」的管理措施，不僅會在隔離檢疫第13天時，發送簡訊提醒民眾自覺有無更多症狀，若曾出現腹瀉、味嗅覺喪失應立即通報，且另外需要再額外自主健康管理7天。
請輸入新聞：0


In [10]:
pre = clf.predict(p)
for i in range(len(pre)):
    print("*" * 30)
    print("這個新聞是：", reverse_trans[pre[i]])
    print("*" * 30)

******************************
這個新聞是： 經濟
******************************


In [11]:
proba_list = clf.predict_proba(p)

for i in range(0, len(proba_list)):
    single = list(zip(trans, proba_list[i]))
    proba_list_single = sorted(single, reverse = True, key = lambda x:x[1])
    for l,pn in proba_list_single:
        print("第",i+1,"則新聞的", "類別：",l, "，機率：", pn)
    print("*" * 30)

第 1 則新聞的 類別： 經濟 ，機率： 0.9494964056330882
第 1 則新聞的 類別： 計算機 ，機率： 0.028252336865462847
第 1 則新聞的 類別： 醫藥 ，機率： 0.021584281687126405
第 1 則新聞的 類別： 交通 ，機率： 0.0003611239782636951
第 1 則新聞的 類別： 環境 ，機率： 0.0002480904587097406
第 1 則新聞的 類別： 軍事 ，機率： 3.4538800937168944e-05
第 1 則新聞的 類別： 教育 ，機率： 2.0929135169803165e-05
第 1 則新聞的 類別： 政治 ，機率： 1.926283579952123e-06
第 1 則新聞的 類別： 藝術 ，機率： 3.671564290275168e-07
第 1 則新聞的 類別： 體育 ，機率： 1.2154164600596626e-12
******************************
